In [1]:
import numpy as np
import random
import heapq

In [2]:
MAX_NUM = 1000

In [45]:
matrix = np.fromfunction(lambda i, j: np.sin(i + j), (MAX_NUM, MAX_NUM))

In [46]:
def target_function(elems_set):
    pos = list(elems_set)
    return np.sum(matrix[np.ix_(pos, pos)])

In [47]:
def crossover(fst_set, snd_set):
    positions = fst_set | snd_set
    elem_2_num = dict(zip(positions, range(len(positions))))
    num_2_elem = dict(zip(range(len(positions)), positions))
    probs = np.zeros(len(positions))
    for e in fst_set:
        probs[elem_2_num[e]] += 1.
    for e in snd_set:
        probs[elem_2_num[e]] += 1.
    probs /= np.sum(probs)
    positions = np.array(list(positions))

    for i in xrange(10):
        yield set(np.random.choice(positions, len(fst_set), replace=False, p=probs))

In [48]:
def mutate(elem_set):
    size = len(elem_set)
    elem_set = elem_set | set(np.random.choice(MAX_NUM, 5, replace=False))
    elem_set = list(elem_set)
    for _ in xrange(5):
        yield set(np.random.choice(elem_set, size, replace=False))

In [49]:
def get_initial_state(k=10, size=100):
    result = []
    for i in xrange(size):
        result.append(set(np.random.choice(MAX_NUM, k, replace=False)))
    return result

In [50]:
#%%time
state = get_initial_state()
scores = [target_function(r) for r in state]

for i in xrange(20):
    print state[0], scores[0]
    print state[1], scores[1]
    print state[2], scores[2]
    print ''
    
    probs = np.array(scores)
    probs -= np.max(probs)
    probs = np.exp(probs / 3.)
    probs /= np.sum(probs)
    
    new_generation = list(state)
    #crossover candidates
    for i in xrange(30):
        fst, snd = np.random.choice(state, 2, replace=False, p=probs)
        for r in crossover(fst, snd):
            new_generation.append(r)
            
    #mutation_candidates
    mutations = []
    for rec in new_generation:
        for r in mutate(rec):
            mutations.append(r)
            
    # selection
    scored = [(x, target_function(x)) for x in new_generation + mutations]
    scored = heapq.nlargest(100, scored, key=lambda x: x[1])
    state, scores = zip(*scored)

set([548, 17, 457, 938, 172, 369, 280, 753, 765, 895]) 10.9176693447
set([963, 748, 496, 881, 979, 478, 376, 761, 670, 575]) 2.55826000781
set([224, 673, 930, 992, 170, 971, 12, 751, 824, 825]) 1.54531416967

set([833, 4, 933, 839, 425, 846, 752, 148, 54, 570]) 81.5626251492
set([997, 167, 425, 205, 305, 306, 181, 570, 827, 764]) 77.0460336098
set([676, 167, 908, 463, 148, 469, 54, 826, 123, 570]) 73.9195063589

set([833, 4, 425, 205, 846, 752, 305, 306, 827, 764]) 92.2304030945
set([4, 997, 167, 425, 205, 846, 752, 54, 570, 827]) 91.7087828328
set([4, 997, 167, 846, 752, 306, 148, 54, 827, 764]) 91.705287997

set([833, 676, 997, 167, 425, 205, 752, 469, 4, 827]) 94.9956156993
set([896, 4, 997, 167, 425, 205, 846, 752, 148, 695]) 94.3112282566
set([833, 4, 997, 167, 425, 205, 752, 148, 54, 827]) 93.3158353661

set([4, 167, 425, 205, 846, 783, 306, 469, 54, 123]) 95.18451709
set([833, 4, 997, 167, 425, 205, 846, 752, 54, 827]) 95.1033760857
set([833, 676, 997, 167, 425, 205, 752, 469, 4

In [52]:
for x in state[0]:
    print x - np.floor(x / 2 / 3.1415) * 2 * 3.1415

3.907
3.851
4.0
4.039
3.87
4.059
3.944
3.887
3.927
4.02


In [28]:
%%time
for i in xrange(1000):
    crossover(
        Recommendation([0, 1, 2, 3], [4]),
        Recommendation([0, 1, 5, 6], [7])
    )

NameError: name 'Recommendation' is not defined

In [254]:
for x in mutate(Recommendation([0, 1, 2, 3], [4])):
    print x

[0, 1, 2, 4] [18]
[0, 1, 2, 18] [4]
[0, 1, 3, 4] [15]
[0, 3, 4, 15] [1]
[1, 2, 3, 10] [4]
[1, 2, 3, 4] [10]


In [352]:
with open('train', 'w') as fo:
    for i, r in enumerate(get_initial_state(100000)):
        fo.write('0\t' + str(target_function(r)) + '\t' + str(i) + '\t0\t')
        positions = np.zeros(20)
        for p in r.small_positions:
            positions[p] = 1
        for p in r.big_positions:
            positions[p] = 2
        fo.write('\t'.join(map(str, positions)))
        fo.write('\n')

In [353]:
mx_model = matrixnet.MatrixnetClient('matrixnet_model/matrixnet.info')

In [354]:
def create_matrix(recs):
    result = []
    for r in recs:
        positions = np.zeros(20)
        for p in r.small_positions:
            positions[p] = 1
        for p in r.big_positions:
            positions[p] = 2
        result.append(positions)    
    return result

In [355]:
import time

In [356]:
%%time
state = get_initial_state()
matrix = create_matrix(state)
scores = mx_model.predict(matrix)
hashes = set(r.unique_hash() for r in state)

for i in xrange(20):
    print state[0], scores[0]
    print state[1], scores[1]
    print state[2], scores[2]
    print ''
    
    probs = np.exp(np.array(scores) / 3.)
    probs /= np.sum(probs)
    
    new_generation = list(state)
    #crossover candidates
    start = time.time()
    for i in xrange(100):
        fst, snd = np.random.choice(state, 2, replace=False, p=probs)
        for r in crossover(fst, snd):
            h = r.unique_hash() 
            if h not in hashes:
                new_generation.append(r)
                hashes.add(h)
            
    print time.time() - start
    start = time.time()
            
    #mutation_candidates
    mutations = []
    for rec in new_generation:
        for r in mutate(rec):
            h = r.unique_hash() 
            if h not in hashes:
                mutations.append(r)
                hashes.add(h)
    
    print time.time() - start
    start = time.time()
            
    # selection
    elems = new_generation + mutations
    matrix = create_matrix(elems)
    scores = mx_model.predict(matrix)

    scored = heapq.nlargest(100, zip(elems, scores), key=lambda x: x[1])
    state, scores = zip(*scored)
    hashes = set(r.unique_hash() for r in state)
    
    print time.time() - start
    start = time.time()

[0, 2, 5, 12] [14] 62.3814826307
[4, 7, 9, 14] [3] 65.8113343082
[3, 4, 5, 10] [14] 63.5426939867

0.181751012802
0.117976903915
0.0310199260712
[16, 17, 18, 19] [10] 114.373404129
[16, 17, 18, 19] [11] 114.199621257
[14, 16, 18, 19] [10] 114.149365041

0.183881998062
0.141770124435
0.0354239940643
[13, 15, 17, 19] [11] 116.337729077
[15, 17, 18, 19] [11] 115.927124725
[16, 17, 18, 19] [10] 114.373404129

0.181839942932
0.105889797211
0.0271098613739
[13, 15, 17, 19] [11] 116.337729077
[13, 15, 17, 19] [9] 116.217838925
[15, 17, 18, 19] [9] 115.928792794

0.180987119675
0.101728916168
0.0262639522552
[13, 15, 17, 19] [11] 116.337729077
[13, 15, 17, 19] [9] 116.217838925
[15, 17, 18, 19] [9] 115.928792794

0.180893182755
0.108633995056
0.0276901721954
[13, 15, 17, 19] [11] 116.337729077
[13, 15, 17, 19] [9] 116.217838925
[15, 17, 18, 19] [9] 115.928792794

0.181003093719
0.108144044876
0.0276830196381
[13, 15, 17, 19] [11] 116.337729077
[13, 15, 17, 19] [9] 116.217838925
[15, 17, 18, 19